# Automated ML

In the cell below, we import all the dependencies that we will need to complete the project.

In [15]:
import joblib
from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.widgets import RunDetails

## Dataset

### Overview

We used heart failure dataset(https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data). Downloaded this dataset from the kaggle website and uploaded to Azure ML Studio and registered it. This data generally contains data about people and has health related data with the death column. Generally, we have to predict early detection of Heart rate failure using ML.

Columns in dataset are - age,	anaemia	creatinine_phosphokinase,	diabetes,	ejection_fraction,	high_blood_pressure	platelets,	serum_creatinine,	serum_sodium,	sex	smoking	time,	DEATH_EVENT.

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.
Reason for ML - Need early detection and management wherein a machine learning model can be of great help.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart-failure-experiment'
experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()
cluster_name = "ComputeCluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except Exception as e:
    print(f"Creating a new compute target (error: {e}")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size = "Standard_DS3_V2",
        min_nodes = 0,
        max_nodes = 4,
    )
    compute_target = ComputeTarget.create(
        ws,
        cluster_name,
        compute_cnfg,
    )
    compute_target.wait_for_completion(
        show_output=True,min_node_count=None,timeout_in_minutes=10,
    )


print(f'compute target: {compute_target.get_status().serialize()}')

Creating a new compute target (error: ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name ComputeCluster not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name ComputeCluster not found in provided workspace"
    }
}
InProgress...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
compute target: {'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-07-09T19:05:31.070000+00:00', 'errors': None, 'creationTime': '2024-07-09T19:05:22.345647+00:00', 'modifiedTime': '2024-07-09T19:05:32.997213+00:00', 'provisioningState': 'Succeed

In [3]:
dataset_name = 'heartfailure'
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [4]:
df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

Field to predict - DEATH_EVENT which is a classification problem, we choose accuracy to be our primary metric here, as data is already registered we have the training dataset ready to use AutoML on it. 

"primary_metric": "accuracy" = This specifies that the primary metric used to evaluate the performance of the models is "accuracy". Accuracy is the ratio of the number of correct predictions to the total number of predictions. It is commonly used for classification tasks where the model needs to predict discrete labels.

"experiment_timeout_minutes": 15 = This sets a time limit for the entire AutoML experiment, which is 15 minutes in this case. The experiment will run for a maximum of 15 minutes. If it doesn't finish within this time, it will stop regardless of whether it has tested all possible configurations or not. This is useful for ensuring experiments do not run indefinitely and helps in managing computational resources.

"max_concurrent_iterations": 5 = This specifies the maximum number of iterations (model training runs) that can be executed concurrently. Running multiple iterations in parallel can speed up the experiment by utilizing available computational resources efficiently. In this case, up to 5 iterations can run simultaneously.

In [7]:
target_column = "DEATH_EVENT"

# automl experiment settings here
automl_settings = {
    "primary_metric": "accuracy",
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    
}

# automl config here
automl_config = AutoMLConfig(
    compute_target= compute_target,
    training_data= dataset,
    task= "classification",
    label_column_name= target_column,
    path= "./automl-run",
    featurization= "auto",
    debug_log= "automl_errors.log",
    enable_early_stopping= True,
    **automl_settings
    )

In [8]:
# submit the experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-experiment,AutoML_d15fa56f-2b6b-4648-92d8-6786b8ede363,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In the cell below, we use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

2024-07-09 19:17:51.620403: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38/lib/python3.9/site-packages/cv2/../../lib64:
2024-07-09 19:17:51.620470: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-experiment,AutoML_d15fa56f-2b6b-4648-92d8-6786b8ede363,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------

{'runId': 'AutoML_d15fa56f-2b6b-4648-92d8-6786b8ede363',
 'target': 'ComputeCluster',
 'status': 'Completed',
 'startTimeUtc': '2024-07-09T19:17:19.767521Z',
 'endTimeUtc': '2024-07-09T19:34:50.722925Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'ComputeCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-experiment","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-262104","workspace_name":"quick-starts-ws-262104","region":"westus2","compute_target":"Compu

## Best Model

Get the best model from the automl experiments and display all the properties of the model.

In [10]:
best_run, best_model = automl_run.get_output()
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()["runDefinition"]["arguments"]

print(f"Best Model Run ID: {best_run.id}\n")
print(f"Best Model Metrics: {best_run_metrics}\n")
print(f"Best Model Accuracy: {best_run_metrics['accuracy']}\n")

2024-07-09 19:35:18.115725: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-09 19:35:23.313920: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-09 19:35:23.314032: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (notebook262104): /proc/driver/nvidia/version does not exist


Best Model Run ID: AutoML_d15fa56f-2b6b-4648-92d8-6786b8ede363_38

Best Model Metrics: {'average_precision_score_macro': 0.90127133237628, 'precision_score_micro': 0.8727586206896552, 'balanced_accuracy': 0.8526649046034057, 'log_loss': 0.35194985644068316, 'recall_score_micro': 0.8727586206896552, 'average_precision_score_micro': 0.9233454587652921, 'AUC_micro': 0.9217858369665741, 'accuracy': 0.8727586206896552, 'precision_score_macro': 0.8589201735320662, 'AUC_macro': 0.9103684604022133, 'precision_score_weighted': 0.8831468766173547, 'weighted_accuracy': 0.8853524375855703, 'norm_macro_recall': 0.7053298092068114, 'f1_score_macro': 0.8476878750460962, 'matthews_correlation': 0.7099962646709519, 'f1_score_micro': 0.8727586206896552, 'f1_score_weighted': 0.8711967419108847, 'AUC_weighted': 0.9103684604022133, 'recall_score_macro': 0.8526649046034057, 'recall_score_weighted': 0.8727586206896552, 'average_precision_score_weighted': 0.9251800674491077, 'accuracy_table': 'aml://artifactI

In [11]:
print(f"More details on best Model {best_model}")

More details on best Model Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook262104/code/Users/odl_user_262104')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('18', Pipeli..., ('randomforestclassifier', RandomForestClassifier(bootstrap=False, criterion='entropy', max_features=0.6, min_samples_leaf=0.01, min_samples_split=0.2442105263157895, n_estimators=200, n_jobs=1))]))], flatten_transform=False, weights=[0.2857142857142857, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]))])


In [12]:
# save the best model
best_run = automl_run.get_best_child()
best_model_name = best_run.properties["model_name"]

In [13]:
best_run.download_files("./outputs")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

We will deploy the autoML model as accuracy is greater than hyperparamter tune.

In [22]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/5c9c9b09/classes.interpret.json',
 'explanation/5c9c9b09/eval_data_viz.interpret.json',
 'explanation/5c9c9b09/expected_values.interpret.json',
 'explanation/5c9c9b09/features.interpret.json',
 'explanation/5c9c9b09/global_names/0.interpret.json',
 'explanation/5c9c9b09/global_rank/0.interpret.json',
 'explanation/5c9c9b09/global_values/0.interpret.json',
 'explanation/5c9c9b09/local_importance_values.interpret.json',
 'explanation/5c9c9b09/per_class_names/0.interpret.json',
 'explanation/5c9c9b09/per_class_rank/0.interpret.json',
 'explanation/5c9c9b09/per_class_values/0.interpret.json',
 'explanation/5c9c9b09/rich_metadata.interpret.json',
 'explanation/5c9c9b09/true_ys_viz.interpret.json',
 'explanation/5c9c9b09/visualization_dict.interpret.json',
 'explanation/5c9c9b09/ys_pred_proba_viz.interpret.json',
 'explanation/5c9c9b09/ys_pred_viz.interpret.json',
 'explanation/90d1a03e/classes.interpret.json',
 'expl

In [28]:
# registering the model
from azureml.core.resource_configuration import ResourceConfiguration
best_model_name = best_run.properties["model_name"]
model = best_run.register_model(
    model_name=best_model_name, 
    model_path="./outputs",
    resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1),
)

print(f"Model name: {model.name}")
print(f"Version: {model.version}")
print(f"RunID: {model.run_id}")

Model name: AutoMLd15fa56f238
Version: 2
RunID: AutoML_d15fa56f-2b6b-4648-92d8-6786b8ede363_38


In [24]:

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

inference_env = Environment.get(ws, "AzureML")


inference_config = InferenceConfig(entry_script="./score.py", environment=inference_env)


service = Model.deploy(ws, "heart-service-1", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)
service.scoring_uri

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-07-09 21:19:58+00:00 Creating Container Registry if not exists.
2024-07-09 21:19:59+00:00 Registering the environment.
2024-07-09 21:19:59+00:00 Use the existing image.
2024-07-09 21:20:00+00:00 Submitting deployment to compute.
2024-07-09 21:20:06+00:00 Checking the status of deployment heart-service-1..
2024-07-09 21:22:53+00:00 Checking the status of inference endpoint heart-service-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


'http://8e870f91-0fe0-4068-ae57-8f1a5c3f65ca.westus2.azurecontainer.io/score'

In [25]:
service.update(enable_app_insights=True)
service.get_logs()

'2024-07-09T21:22:47,653832157+00:00 - rsyslog/run \n2024-07-09T21:22:47,658766026+00:00 - gunicorn/run \n2024-07-09T21:22:47,661244571+00:00 | gunicorn/run | \n2024-07-09T21:22:47,662312054+00:00 | gunicorn/run | ###############################################\n2024-07-09T21:22:47,664714364+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-07-09T21:22:47,668404615+00:00 | gunicorn/run | ###############################################\n2024-07-09T21:22:47,670773083+00:00 | gunicorn/run | \n2024-07-09T21:22:47,671408757+00:00 - nginx/run \n2024-07-09T21:22:47,676406848+00:00 | gunicorn/run | \n2024-07-09T21:22:47,682659331+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240624.v1\n2024-07-09T21:22:47,683991325+00:00 | gunicorn/run | \n2024-07-09T21:22:47,686520892+00:00 | gunicorn/run | \n2024-07-09T21:22:47,688285764+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

In [27]:
import urllib.request
import json
import os

data =  {
  "data": [
    {
      "age": 18.0,
      "anaemia": 0,
      "creatinine_phosphokinase": 0,
      "diabetes": 1,
      "ejection_fraction": 0,
      "high_blood_pressure": 1,
      "platelets": 0.0,
      "serum_creatinine": 0.0,
      "serum_sodium": 0,
      "sex": 1,
      "smoking": 0,
      "time": 0
    }
  ]
}

body = str.encode(json.dumps(data))

url = 'http://8e870f91-0fe0-4068-ae57-8f1a5c3f65ca.westus2.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[1]'


In [29]:
service.delete()
compute_target.delete()

Running
2024-07-09 21:46:39+00:00 Check and wait for operation (af73b200-8ead-45d3-a131-1580acb56fd8) to finish.
2024-07-09 21:46:42+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
